In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# "moons", "circles", "xor", "spirals", "rings", "checker"
DATASET = "moons"

N = 600
rng = np.random.default_rng(0)

def make_xor(n=600, noise=0.2, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    X = rng.uniform(-1, 1, size=(n, 2))
    y = (X[:, 0] * X[:, 1] > 0).astype(int)
    X += rng.normal(0, noise, size=X.shape)
    return X, y

def make_spirals(n=600, noise=0.2, rng=None, turns=3):
    if rng is None:
        rng = np.random.default_rng()
    n2 = n // 2
    theta = np.linspace(0, turns * 2*np.pi, n2)
    r = np.linspace(0.0, 1.0, n2)
    x1 = np.c_[r*np.cos(theta), r*np.sin(theta)]
    x2 = np.c_[r*np.cos(theta + np.pi), r*np.sin(theta + np.pi)]
    X = np.vstack([x1, x2]) + rng.normal(0, noise, size=(2*n2, 2))
    y = np.hstack([np.zeros(n2, dtype=int), np.ones(n2, dtype=int)])
    return X, y

def make_rings(n=600, rng=None, noise=0.1):
    if rng is None:
        rng = np.random.default_rng()
    n2 = n // 2
    # inner ring radius ~ 0.5, outer ~ 1.0
    angles1 = rng.uniform(0, 2*np.pi, n2)
    angles2 = rng.uniform(0, 2*np.pi, n2)
    r1 = rng.normal(0.5, noise, n2)
    r2 = rng.normal(1.0, noise, n2)
    x1 = np.c_[r1*np.cos(angles1), r1*np.sin(angles1)]
    x2 = np.c_[r2*np.cos(angles2), r2*np.sin(angles2)]
    X = np.vstack([x1, x2])
    y = np.hstack([np.zeros(n2, dtype=int), np.ones(n2, dtype=int)])
    return X, y

def make_checker(n=600, rng=None, noise=0.1, tiles=4):
    if rng is None:
        rng = np.random.default_rng()
    s = int(np.sqrt(n))
    x1 = np.linspace(-1, 1, s)
    x2 = np.linspace(-1, 1, s)
    XX, YY = np.meshgrid(x1, x2)
    X = np.c_[XX.ravel(), YY.ravel()]
    y = (((np.floor((XX + 1)*tiles/2) + np.floor((YY + 1)*tiles/2)) % 2) > 0).astype(int).ravel()
    # jitter
    X += rng.normal(0, noise, size=X.shape)
    # sample n points
    idx = rng.choice(len(X), size=n, replace=True)
    return X[idx], y[idx]

def load_dataset(kind=DATASET, n=N, rng=rng):
    if kind == "moons":
        X, y = make_moons(n_samples=n, noise=0.25, random_state=0)
    elif kind == "circles":
        X, y = make_circles(n_samples=n, noise=0.1, factor=0.5, random_state=0)
    elif kind == "xor":
        X, y = make_xor(n=n, noise=0.15, rng=rng)
    elif kind == "spirals":
        X, y = make_spirals(n=n, noise=0.08, rng=rng, turns=3)
    elif kind == "rings":
        X, y = make_rings(n=n, rng=rng, noise=0.05)
    elif kind == "checker":
        X, y = make_checker(n=n, rng=rng, noise=0.03, tiles=6)
    else:
        raise ValueError("Unknown dataset name.")
    return X, y

def plot_decision_function(ax, model, X, y, title):
    # mesh
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 400),
        np.linspace(y_min, y_max, 400)
    )
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # decision boundary & margins
    cs = ax.contour(xx, yy, Z, levels=[-1, 0, 1], linestyles=["--", "-", "--"])
    ax.clabel(cs, inline=1, fontsize=8)
    # points
    ax.scatter(X[:,0], X[:,1], c=y, s=15, alpha=0.7, edgecolor="none")
    ax.set_title(title)

# Load chosen dataset
X, y = load_dataset(DATASET, N, rng)

# Models to compare
models = [
    ("Linear SVM", make_pipeline(StandardScaler(), SVC(kernel="linear", C=1.0))),
    ("Poly deg=3", make_pipeline(StandardScaler(), SVC(kernel="poly", degree=3, C=5.0, coef0=1.0, gamma="scale"))),
    ("RBF γ=scale", make_pipeline(StandardScaler(), SVC(kernel="rbf", C=5.0, gamma="scale"))),
    ("RBF γ=auto",  make_pipeline(StandardScaler(), SVC(kernel="rbf", C=5.0, gamma="auto"))),
    ("RBF (smooth)", make_pipeline(StandardScaler(), SVC(kernel="rbf", C=1.0, gamma=0.5))),
    ("RBF (wiggly)", make_pipeline(StandardScaler(), SVC(kernel="rbf", C=50.0, gamma=5.0))),
]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()

for ax, (name, clf) in zip(axes, models):
    clf.fit(X, y)
    plot_decision_function(ax, clf, X, y, name)

plt.suptitle(f"Kernel SVMs on '{DATASET}' dataset")
plt.tight_layout()
plt.show()